In [1]:
using ITensors, ITensorMPS
using LinearAlgebra
using Plots

for N in 1:1
    s = siteinds("Qubit",N)
    
    
    gates = ITensor[]
    
    function H_gate(x)
        Hadamard = op("H",s[x]) # op  is  a function which converts matrices to ITensors
        
    end
    
    function swap_gate(i,j)
        swap = [ 1 0 0 0 ; 0 0 1 0; 0 1 0 0; 0 0 0 1] 
        swap_gate = op(swap, s[i] , s[j])
        
    end
    
    function phase_gate1(i,j,θ)
        phase = [ 1 0 0 0 ; 0 1 0 0; 0 0 1 0; 0 0 0 exp(-1*im * θ)] 
        phase_gate = op(phase, s[i] , s[j])
        # s[i] and s[j] are the control qubit 
        
    end
    
    # Define the QFT MPO
    function qft_mpo(N::Int, sites)
        # Initialize MPO with sites
        W = MPO(sites,"I")  
        
        cutoff=1e-18
        # Apply Hadamard and phase gates to build the QFT MPO
        
        
        for i in 1:N
            W = apply(op("H",sites[N+1-i]),W,cutoff=cutoff)
    
            for j in i+1:N
                θ = π / 2^(j-i)
                P = phase_gate1(N+1-i,N+1-j,θ)
                # Update MPO by combining with phase gate tensor
                W = apply(P,W,cutoff=cutoff)
    
    
            end
                
        end
        
        return W
    end
    
    W = qft_mpo(N, s)
    println(maxlinkdim(W))
    orthogonalize!(W, 1)   # Orthogonalize W at the first index
    
    using HDF5
    
    f3 = h5open("Basis states Final/MPO_$N.h5","w")
    write(f3,"W", W)
    close(f3)
    
    f4 = h5open("Basis states Final/MPO_$N.h5","r")
    W1 = read(f4,"W",MPO)
    close(f4)
end

1


In [148]:
# # Taking the input 

# x = range(0, stop=2π, length=2^N)
# input_array = cos.(x)

# array = input_array / norm(input_array) # Input
# ITensors.disable_warn_order()

# cutoff1 = 1E-18
# maxdim1 = 10
# T = ITensor(array,s)
# ψ = MPS(T,s;cutoff=cutoff1,maxdim=maxdim1)

# println(maxlinkdim(ψ))

In [149]:
# function generate_basis_states(n)
#     basis_states = []
#     for i in 0:(2^n - 1)
#         binary_str = bitstring(i)[end-n+1:end]  # Get last n bits of bitstring
#         push!(basis_states, collect(binary_str))  # Collect splits string into array of characters
#     end
#     return basis_states
# end

# MPS_states = []
# basis_states = generate_basis_states(N)

# for i in 1:(2^N)
#     push!(MPS_states, MPS(s,string.(basis_states[i])))
# end

# using HDF5
# f = 0 
# for (i, mps) in enumerate(MPS_states)
#     f = h5open("Basis states Final/MPS_$N/MPS_create_$i.h5","w")
#     write(f,"M", mps)
# end
# close(f)
